In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-11-25"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
14738,2024-11-25,Eua Ncaa,14:30,La Lafayette,UAB,4.67,1.21,156.5,1.91,1.80,9.5,1.91,1.91,Or3jUSuR,0.214133,0.826446,0.523560,0.555556,0.040579,218.812,173.707437,0.793866,0.6,1.341641,2.236068,223.27,3.158,2.462290,0.779699,-64.0,125.146,50.875347,0.406528,1.2,1.643168,1.369306,112.88,1.620,0.945489,0.583635,11.0,83,83,2.69,1.36,127.332,206.638,0.832173,0.041931,0.000000,0.15,0.030,122.333333,0.704644,0.6,-0.104644,-0.96,-0.192,-1.093750,0.432120,0.5,0.067880
14739,2024-11-25,Eua Ncaa,19:00,Old Dominion,High Point,6.90,1.12,145.5,1.91,1.91,12.5,1.91,1.91,xh8e6Pz8,0.144928,0.892857,0.523560,0.523560,0.037785,167.526,70.217862,0.419146,0.0,0.000000,NaN,254.28,2.632,1.370774,0.520811,-110.0,88.210,11.717760,0.132839,1.8,1.643168,0.912871,76.86,1.144,0.149766,0.130915,56.0,52,61,4.89,1.26,153.080,123.058,1.019221,0.000000,0.000000,-3.57,-0.714,-8.263305,0.562338,0.2,-0.362338,-0.31,-0.062,-1.935484,0.616187,0.7,0.083813
14740,2024-11-25,Eua Ncaa,18:00,CS Northridge,Denver,1.37,3.29,156.5,1.91,1.91,-7.5,1.95,1.72,8Sjj42Qp,0.729927,0.303951,0.523560,0.523560,0.033878,199.900,57.057433,0.285430,1.8,1.643168,0.912871,109.47,2.690,1.224234,0.455106,19.0,476.878,407.989544,0.855543,0.6,1.341641,2.236068,358.43,7.246,6.359027,0.877591,-47.0,89,73,1.23,4.91,157.044,476.878,0.582680,0.000000,0.088629,-3.77,-0.754,-0.490716,0.565562,0.4,-0.165562,-3.05,-0.610,-3.754098,0.309048,0.2,-0.109048
14741,2024-11-25,Eua Ncaa,17:00,George Washington,Illinois State,1.91,1.91,148.5,1.80,1.91,-1.5,2.00,1.73,636FO63l,0.523560,0.523560,0.555556,0.523560,0.047120,135.758,91.139021,0.671334,1.8,1.643168,0.912871,111.00,1.804,1.345188,0.745669,29.0,198.154,79.259838,0.399991,1.8,1.643168,0.912871,279.72,2.588,1.004674,0.388205,10.0,74,84,1.50,3.33,155.924,393.010,0.000000,0.041931,0.102369,1.80,0.360,2.527778,0.588311,0.5,-0.088311,10.97,2.194,0.414768,0.335567,0.4,0.064433
14742,2024-11-25,Eua Ncaa,17:00,Alabama A&M,IU Indy,1.55,2.56,143.5,1.91,1.91,-4.5,2.00,1.71,IVwFQAGt,0.645161,0.390625,0.523560,0.523560,0.035786,162.564,16.172937,0.099487,1.2,1.643168,1.369306,140.00,2.336,0.275826,0.118076,2.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,70,57,2.00,2.22,138.150,0.000,0.347532,0.000000,0.110545,0.94,0.188,2.925532,0.541821,0.6,0.058179,0.00,0.000,inf,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14817,2024-11-25,Eua Ncaa,22:30,South Carolina,Xavier,2.99,1.37,0.0,0.00,0.00,6.5,1.80,1.91,CrE8JXbA,0.334448,0.729927,inf,inf,0.064375,118.232,53.942260,0.456241,1.2,1.643168,1.369306,86.52,1.740,1.160043,0.666691,-14.0,223.100,206.587931,0.925988,1.2,1.643168,1.369306,78.78,3.376,3.619051,1.071994,-26.0,84,78,1.03,1.01,99.452,262.404,0.525465,NaN,0.041931,-1.57,-0.314,-6.337580,0.736170,0.6,-0.136170,-1.80,-0.360,-1.027778,0.432541,0.4,-0.032541
14818,2024-11-25,Eua Ncaa,22:30,San Jose State,UTEP,2.61,1.51,0.0,0.00,0.00,4.5,1.83,1.83,bifhv9jA,0.383142,0.662252,inf,inf,0.045393,382.574,363.563404,0.950309,0.0,0.000000,NaN,782.00,5.890,5.610018,0.952465,-52.0,179.840,38.909333,0.216355,1.8,1.6431

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
14741,17:00,Eua Ncaa,George Washington,Illinois State,1.91,Back Home
14746,07:00,Turquia Tkbl Feminina,Mersin Gelisim F,Urla Bld F,2.20,Back Home
14757,23:00,Eua Ncaa,Abilene Christian,Southern Miss,1.69,Back Home
14759,12:00,Turquia Kbsl Feminina,Besiktas F,Galatasaray F,1.85,Back Home
14771,21:30,Eua Ncaa,Boston College,Missouri State,1.69,Back Home
14784,08:30,China Wcba Feminina,Sichuan F,Fujian F,1.74,Back Home
